In [ ]:
!pip install -r requirements.txt

In [ ]:
import zipfile
import os

# 업로드된 ZIP 파일 압축 해제
with zipfile.ZipFile("/content/archive.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

# 압축 해제 결과 확인
os.listdir("/content/caltech-101")[:5]

['octopus', 'platypus', 'dollar_bill', 'dolphin', 'buddha']

In [ ]:
"""
프로그램 파이프라인 개요:
    1) 데이터: Kaggle Caltech-101 압축을 해제한 폴더에서 Faces_easy 클래스를 제외한다.
       (Caltech-101은 101개 객체 클래스 + background로 구성된 대표적인 분류 데이터셋)
    2) 입력: 모든 이미지를 128×128로 리사이즈하고 RGB를 0~1로 정규화한다. (Keras/TensorFlow 권장 전처리)
    3) 분할: 전체를 무작위 셔플 후 train/val/test = 8:1:1로 분할한다.
    4) 증강: dataset에 수평 뒤집기, 회전, 확대/축소, 대비 변화를 적용한다.
       (Keras 이미지 증강 레이어 사용; 추론 시에는 비활성)
    5) 탐색: 필요 시 하이퍼파라미터 탐색을 수행한다.
       - 구조 탐색: conv_blocks, base_filters 조합
       - 세부 탐색: dropout, optimizer, learning rate, epochs 조합
       - 조기종료(EarlyStopping)와 Plateau 시 학습률 감소(ReduceLROnPlateau)로 안정화
         (val_loss 모니터링, patience 설정)
       - 검증 성능(F1 포함)을 기준으로 최적 조합을 선택한다.
    6) 최종학습: 선택된 최적 하이퍼파라미터로 train/val 전체를 활용해 다시 학습하고
       가중치를 best_model.weights.h5로 저장한다. (가중치 형식)
    7) 평가: test 데이터로 예측을 수행하고 confusion_matrix.png와 매크로 F1 점수를 출력한다.

파일 구조:
    - data_loader.py  : 데이터 로딩/전처리/분할(Faces_easy 제거, 8:1:1, cache/prefetch)
    - cnn_model.py    : 모델 빌드/증강/탐색/학습/평가 유틸
    - train.py        : 학습 실행 스크립트 (필요 시 --search로 하이퍼파라미터 탐색)
    - evaluate.py     : 평가 실행 스크립트 (학습 때의 구조/파라미터를 동일하게 지정해야함)

Colab 사용 절차:
    1) 로컬에서 Kaggle Caltech-101(archive.zip) 다운로드 후 Colab ‘파일’ 패널로 드래그 (/content 내로)
    2) 런타임 유형: GPU(T4) 설정
    3) 라이브러리 설치:
    !pip install tensorflow tensorflow-datasets tensorflow.keras scikit-learn matplotlib pathlib tqdm
    !pip install --upgrade tensorflow
    4) 데이터 압축 해제 후 경로 예: /content/caltech-101
    5) 학습 실행 예시:
       %cd ML_CNN
       !python train.py --data-dir /content/caltech-101 --augment
       # 하이퍼파라미터 탐색을 켜려면: --search
    6) 평가 실행 예시:
       !python evaluate.py --data-dir /content/caltech-101 \
           --weights-path best_model.weights.h5 \
           --conv-blocks 4 --base-filters 64 --dropout 0.3 \
           --optimizer Adam --lr 0.001
    7) 생성물 확인:
       - results/            : 탐색 시 조합별 *.weights.h5 보관
       - best_model.weights.h5, confusion_matrix.png : 루트에 생성
"""

'\n프로그램 파이프라인 개요:\n    1) 데이터: Kaggle Caltech-101 압축을 해제한 폴더에서 Faces_easy 클래스를 제외한다.\n       (Caltech-101은 101개 객체 클래스 + background로 구성된 대표적인 분류 데이터셋)\n    2) 입력: 모든 이미지를 128×128로 리사이즈하고 RGB를 0~1로 정규화한다. (Keras/TensorFlow 권장 전처리)\n    3) 분할: 전체를 무작위 셔플 후 train/val/test = 8:1:1로 분할한다.\n    4) 증강: dataset에 수평 뒤집기, 회전, 확대/축소, 대비 변화를 적용한다.\n       (Keras 이미지 증강 레이어 사용; 추론 시에는 비활성)\n    5) 탐색: 필요 시 하이퍼파라미터 탐색을 수행한다.\n       - 구조 탐색: conv_blocks, base_filters 조합\n       - 세부 탐색: dropout, optimizer, learning rate, epochs 조합\n       - 조기종료(EarlyStopping)와 Plateau 시 학습률 감소(ReduceLROnPlateau)로 안정화\n         (val_loss 모니터링, patience 설정)\n       - 검증 성능(F1 포함)을 기준으로 최적 조합을 선택한다.\n    6) 최종학습: 선택된 최적 하이퍼파라미터로 train/val 전체를 활용해 다시 학습하고\n       가중치를 best_model.weights.h5로 저장한다. (가중치 형식)\n    7) 평가: test 데이터로 예측을 수행하고 confusion_matrix.png와 매크로 F1 점수를 출력한다.\n\n파일 구조:\n    - data_loader.py  : 데이터 로딩/전처리/분할(Faces_easy 제거, 8:1:1, cache/prefetch)\n    - cnn_model.py    : 모델 빌드/증강/탐색/학습/평가 유틸\n    - train.p

#Code run

In [ ]:
!python cnn_model.py

2025-08-31 07:21:57.701381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756624917.733674    3185 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756624917.743633    3185 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756624917.766471    3185 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756624917.766510    3185 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756624917.766517    3185 computation_placer.cc:177] computation placer alr

In [ ]:
!python data_loader.py

2025-08-31 07:27:40.757243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756625260.788870    4672 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756625260.796245    4672 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756625260.818640    4672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625260.818679    4672 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625260.818687    4672 computation_placer.cc:177] computation placer alr

In [ ]:
!python train.py --data-dir /content/caltech-101 --augment

2025-08-31 07:33:05.723059: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756625585.741572    6059 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756625585.747137    6059 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756625585.761538    6059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625585.761564    6059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756625585.761568    6059 computation_placer.cc:177] computation placer alr

In [14]:
!python evaluate.py --data-dir /content/caltech-101 --weights-path best_model.weights.h5 \
                       --conv-blocks 4 --base-filters 64 --dropout 0.3 \
                       --optimizer Adam --lr 1e-3

2025-08-31 07:52:36.675574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756626756.694711   11403 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756626756.700613   11403 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756626756.715656   11403 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756626756.715691   11403 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756626756.715695   11403 computation_placer.cc:177] computation placer alr